In [1]:
import pandas as pd
import numpy as np

In [2]:
labels = pd.read_csv("data-science-bowl-2019/train.csv")
target = pd.read_csv("data-science-bowl-2019/train_labels.csv")

In [3]:
groupped_sessions = labels.groupby(["installation_id", "game_session"]).agg(
    {"title":"nunique",
    "game_time":"max",
    "event_code":"nunique",
    "event_id":["nunique", "count"]}).reset_index()
groupped_sessions.columns = ["installation_id", "game_session", "titles", 
                             "max_game_time", "codes_number", "events_number", "activities_number"]
groupped_sessions.head()

,installation_id,game_session,titles,max_game_time,codes_number,events_number,activities_number
0,0001e90f,04168b61c0650977,1,0,1,1,1
1,0001e90f,07bacda7f9437b38,1,249720,16,16,142
2,0001e90f,0848ef14a8dc6892,1,194335,9,9,267
3,0001e90f,091587ed87b9637c,1,0,1,1,1
4,0001e90f,17eeb7f223665f53,1,0,1,1,1


In [4]:
train_dataset = pd.merge(left = groupped_sessions, right = target, how="inner", on = ["installation_id", "game_session"])
train_dataset.head()

,installation_id,game_session,titles,max_game_time,codes_number,events_number,activities_number,title,num_correct,num_incorrect,accuracy,accuracy_group
0,0006a69f,6bdf9623adc94d89,1,26827,15,15,35,Mushroom Sorter (Assessment),1,0,1.0,3
1,0006a69f,77b8ee947eb84b4e,1,92799,10,10,87,Bird Measurer (Assessment),0,11,0.0,0
2,0006a69f,901acc108f55a5a1,1,39803,16,16,48,Mushroom Sorter (Assessment),1,0,1.0,3
3,0006a69f,9501794defd84e4d,1,31843,18,18,42,Mushroom Sorter (Assessment),1,1,0.5,2
4,0006a69f,a9ef3ecb3d1acc6a,1,36368,14,14,32,Bird Measurer (Assessment),1,0,1.0,3


In [5]:
len(train_dataset)

17690

In [48]:
del train_dataset["num_correct"]
del train_dataset["num_incorrect"]
del train_dataset["accuracy"]

In [50]:
del train_dataset["game_session"]
del train_dataset["titles"]

In [ ]:
del train_dataset["inst"]

In [51]:
results = train_dataset["accuracy_group"]
del train_dataset["accuracy_group"]
train_dataset = pd.get_dummies(train_dataset)
train_dataset.head()

,max_game_time,codes_number,events_number,activities_number,installation_id_0006a69f,installation_id_0006c192,installation_id_00129856,installation_id_001d0ed0,installation_id_00225f67,installation_id_00279ac5,...,installation_id_ff9305d7,installation_id_ff9715db,installation_id_ffc90c32,installation_id_ffd2871d,installation_id_ffeb0b1b,title_Bird Measurer (Assessment),title_Cart Balancer (Assessment),title_Cauldron Filler (Assessment),title_Chest Sorter (Assessment),title_Mushroom Sorter (Assessment)
0,26827,15,15,35,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,92799,10,10,87,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,39803,16,16,48,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,31843,18,18,42,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,36368,14,14,32,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [69]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5)
scores = []
importances = []
# do not forget to suffle the data
for train, test in kf.split(train_dataset):
    X_train, X_test = train_dataset.iloc[train], train_dataset.iloc[test]
    Y_train, Y_test = results.iloc[train], results.iloc[test]
    rf = RandomForestRegressor(n_estimators=1000, random_state=42).fit(X_train, Y_train)
    importances.append(rf.feature_importances_)

[ 3538  3539  3540 ... 17687 17688 17689]
[    0     1     2 ... 17687 17688 17689]


KeyboardInterrupt: 

In [55]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble.forest:

class RandomForestRegressor(ForestRegressor)
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and use averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="mse")
 |      The function to measure the quality of a split. Supported criteria
 |      are "mse" for the mean squared error, which is equal to variance
 |      reduction as feature selection criterion, and "mae" for the mean
 |      absolute 

In [53]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(train_set, train_labels)

C:\Users\Student\Anaconda3\envs\kaggle\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [54]:
predictions = rf.predict(test_set)

In [49]:
train_dataset.head()

,installation_id,game_session,titles,max_game_time,codes_number,events_number,activities_number,title,accuracy_group
0,0006a69f,6bdf9623adc94d89,1,26827,15,15,35,Mushroom Sorter (Assessment),3
1,0006a69f,77b8ee947eb84b4e,1,92799,10,10,87,Bird Measurer (Assessment),0
2,0006a69f,901acc108f55a5a1,1,39803,16,16,48,Mushroom Sorter (Assessment),3
3,0006a69f,9501794defd84e4d,1,31843,18,18,42,Mushroom Sorter (Assessment),2
4,0006a69f,a9ef3ecb3d1acc6a,1,36368,14,14,32,Bird Measurer (Assessment),3


In [14]:
test = pd.read_csv("data-science-bowl-2019/test.csv")

In [15]:
test[(test["type"] == "Assessment")].nunique()

event_id              94
game_session        3347
timestamp          99954
event_data         95546
installation_id     1000
event_count          447
event_code            22
game_time          47724
title                  5
type                   1
world                  3
dtype: int64

In [27]:
labels[(labels["type"]== "Assessment") & 
       ((labels["event_code"] == 4100) | (labels["event_code"] == 4110))]["game_session"].nunique()

18556

In [17]:
labels[(labels["type"]== "Assessment") & (labels["event_code"] == 2010)]["event_data"].to_csv("descriptions.csv", index=None)

C:\Users\Student\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
assessments_sessions = labels.loc[labels["type"]== "Assessment", "game_session"]

In [11]:
assessments_sessions.nunique()

21239

In [3]:
labels["timestamp"] = pd.to_datetime(labels["timestamp"], format = "%Y-%m-%d %H:%M:%S")

NameError: name 'df' is not defined